In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import AmazonGoogleBenchmark

benchmark = AmazonGoogleBenchmark(data_dir_path="../data/")
benchmark

14:19:20 INFO:Extracting Amazon-Google...
14:19:20 INFO:Reading Amazon-Google row_dict...
14:19:20 INFO:Reading Amazon-Google train.csv...
14:19:20 INFO:Reading Amazon-Google valid.csv...
14:19:20 INFO:Reading Amazon-Google test.csv...


<AmazonGoogleBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/amazon_google_exp_data.zip

## Preprocess

In [5]:
attr_list = ['title', 'manufacturer', 'price']

In [6]:
from tqdm.auto import tqdm
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

  0%|          | 0/4589 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [9]:
attr_config_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'semantic_title': {
        'source_attr': 'title',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
    },
    'manufacturer': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [10]:
from entity_embed import AttrConfigDictParser

row_numericalizer = AttrConfigDictParser.from_dict(attr_config_dict, row_list=benchmark.row_dict.values())
row_numericalizer.attr_config_dict

14:19:20 INFO:For attr=title, computing actual max_str_len
14:19:20 INFO:For attr=title, using actual_max_str_len=26
14:19:20 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
14:19:23 INFO:For attr=manufacturer, computing actual max_str_len
14:19:23 INFO:actual_max_str_len=15 must be even to enable NN pooling. Updating to 16
14:19:23 INFO:For attr=manufacturer, using actual_max_str_len=16
14:19:23 INFO:For attr=price, computing actual max_str_len
14:19:23 INFO:actual_max_str_len=9 must be even to enable NN pooling. Updating to 10
14:19:23 INFO:For attr=price, using actual_max_str_len=10


{'title': AttrConfig(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=26, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True),
 'semantic_title': AttrConfig(source_attr='title', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%

In [11]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=32,
    eval_batch_size=128,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import Matcher

model = Matcher(
    row_numericalizer=row_numericalizer,
)

In [13]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStoppingMinEpochs(
   monitor='valid_f1_at_0.5',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_save_dir = '../tb_logs'
tb_name = f'matcher-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_save_dir, name=tb_name),
    reload_dataloaders_every_epoch=True
)

GPU available: True, used: True
14:19:23 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
14:19:23 INFO:TPU available: None, using: 0 TPU cores


In [14]:
trainer.fit(model, datamodule)

14:19:23 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 7.2 M 
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
5.5 M     Trainable params
1.7 M     Non-trainable params
7.2 M     Total params
28.779    Total estimated model params size (MB)
14:19:26 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 7.2 M 
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
5.5 M     Trainable params
1.7 M     Non-trainable params
7.2 M     Total params
28.779    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

## Testing

In [15]:
trainer.test(ckpt_path='best', verbose=False)

14:30:41 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.6091631054878235,
  'test_f1_at_0.4': 0.6177487373352051,
  'test_f1_at_0.5': 0.6092062592506409,
  'test_precision_at_0.3': 0.5337620377540588,
  'test_precision_at_0.4': 0.5633803009986877,
  'test_precision_at_0.5': 0.5735849142074585,
  'test_recall_at_0.3': 0.7093713879585266,
  'test_recall_at_0.4': 0.6837314963340759,
  'test_recall_at_0.5': 0.649544894695282}]